In [ ]:
# 1. Установка micromamba
!wget -qO- https://micromamba.snakepit.net/api/micromamba/linux-64/latest | tar -xvj bin/micromamba

# 2. Создание окружения и установка инструментов
!./bin/micromamba create -y -n bio \
  -c bioconda -c conda-forge \
  fastqc trimmomatic bwa samtools bcftools fastp hisat2 bowtie2 bedtools seqtk multiqc

# 3. Активация окружения и проверка (для примера)
!./bin/micromamba run -n bio fastqc --help
!./bin/micromamba run -n bio bwa
!./bin/micromamba run -n bio samtools --version

bin/micromamba
[+] 0.0s
[+] 0.1s
bioconda/linux-64 ..  ⣾  [+] 0.0s
[+] 0.1s
bioconda/linux-64     ⣾  
bioconda/noarch       ⣾  
conda-forge/linux-64   1%
conda-forge/noarch    ⣾  [+] 0.2s
bioconda/linux-64     33%
bioconda/noarch       24%
conda-forge/linux-64   8%
conda-forge/noarch     7%[+] 0.3s
bioconda/linux-64     75%
bioconda/noarch       70%
conda-forge/linux-64  14%
conda-forge/noarch    19%[+] 0.4s
bioconda/linux-64     93%
bioconda/noarch      100%
conda-forge/linux-64  16%
conda-forge/noarch    27%bioconda/linux-64                                 
bioconda/noarch                                   
[+] 0.5s
conda-forge/linux-64  23%
conda-forge/noarch    40%[+] 0.6s
conda-forge/linux-64  32%
conda-forge/noarch    58%[+] 0.7s
conda-forge/linux-64  40%
conda-forge/noarch    66%[+] 0.8s
conda-forge/linux-64  44%
conda-forge/noarch    85%[+] 0.9s
conda-forge/linux-64  49%
conda-forge/noarch    97%[+] 1.0s
conda-forge/linux-64  50%
conda-forge/noarch    97%conda-forge/noarch     

# СКАЧИВАЕМ РЕФЕРЕНСНЫЙ ГЕНОМ ( **Escherichia coli**) через NCBI Datasets
datasets accession GCF_000005845.2 --include gff3,rna,cds,protein,genome,seq-report

https://www.ncbi.nlm.nih.gov/datasets/genome/GCF_000005845.2/

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Escherichia_coli/reference/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
!gunzip GCF_000005845.2_ASM584v2_genomic.fna.gz
!mv GCF_000005845.2_ASM584v2_genomic.fna ecoli.fna


--2025-05-07 13:40:13--  https://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Escherichia_coli/reference/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 130.14.250.13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  3.80MB/s    in 0.3s    

2025-05-07 13:40:13 (3.80 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]



# Cкачиваем риды (NGS-данные) для Escherichia coli из NCBI

https://www.ncbi.nlm.nih.gov/sra/SRX1317390[accn]

In [10]:
!apt-get install -y sra-toolkit
!fastq-dump --split-files SRR2584863

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
Suggested packages:
  blends-doc menu-l10n gksu | kde-runtime | ktsuss
The following NEW packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
  sra-toolkit
0 upgraded, 7 newly installed, 0 to remove and 34 not upgraded.
Need to get 8,290 kB of archives.
After this operation, 23.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 menu amd64 2.1.47ubuntu4 [354 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 blends-common all 0.7.4ubuntu1 [15.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libkdf5-2 amd64 2.11.2+dfsg-4build2 [14.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libncbi-vdb2 amd64 2.11.2+dfsg-4build2 [1,364 kB]
Get

#Проверяем качество ридов триммируем

In [ ]:
# Установка FastQC (если ещё не установлен)
!if [ ! -f "/usr/local/bin/fastqc" ]; then \
    wget -O fastqc.zip https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.12.1.zip && \
    unzip -o fastqc.zip && \
    chmod +x FastQC/fastqc && \
    ln -s $(pwd)/FastQC/fastqc /usr/local/bin/fastqc ; \
else echo "FastQC уже установлен"; fi

# Установка fastp v0.23.4
!wget http://opengene.org/fastp/fastp.0.23.4 -O fastp
!chmod +x fastp
!sudo mv fastp /usr/local/bin/

# Проверка версий
!fastqc --version
!fastp --version

# Запуск FastQC
!fastqc SRR2584863_1.fastq SRR2584863_2.fastq

# Тримминг fastp
!fastp \
  -i SRR2584863_1.fastq \
  -I SRR2584863_2.fastq \
  -o clean_1.fastq \
  -O clean_2.fastq \
  -h fastp_report.html \
  -j fastp_report.json


FastQC уже установлен
--2025-05-07 14:00:36--  http://opengene.org/fastp/fastp.0.23.4
Resolving opengene.org (opengene.org)... 8.210.133.117
Connecting to opengene.org (opengene.org)|8.210.133.117|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://opengene.org/fastp/fastp.0.23.4 [following]
--2025-05-07 14:00:37--  https://opengene.org/fastp/fastp.0.23.4
Connecting to opengene.org (opengene.org)|8.210.133.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10946176 (10M) [application/octet-stream]
Saving to: ‘fastp’

fastp               100%[===================>]  10.44M  4.55MB/s    in 2.3s    

2025-05-07 14:00:40 (4.55 MB/s) - ‘fastp’ saved [10946176/10946176]

FastQC v0.12.1
fastp 0.23.4
null
null
Started analysis of SRR2584863_1.fastq
Approx 5% complete for SRR2584863_1.fastq
Approx 10% complete for SRR2584863_1.fastq
Approx 15% complete for SRR2584863_1.fastq
Approx 20% complete for SRR2584863_1.fastq
Approx

In [ ]:
!fastp --version


fastp 0.23.4


In [ ]:
#Проверяем создались ли файлы ридов
!ls -lh SRR2584863_*.fastq

-rw-r--r-- 1 root root 663M May  7 13:41 SRR2584863_1.fastq
-rw-r--r-- 1 root root 663M May  7 13:41 SRR2584863_2.fastq


In [ ]:
#Проверяем создались ли очищенные после тримменга файлы ридов
!ls -lh clean_*.fastq

-rw-r--r-- 1 root root 538M May  7 14:01 clean_1.fastq
-rw-r--r-- 1 root root 538M May  7 14:01 clean_2.fastq


# Выравнивание ридов по генам в формате .bam

In [ ]:
#Установка bwa и samtools
!apt-get install -y bwa samtools
#Индексация референсного генома
!bwa index ecoli.fna
#Выравнивание ридов dв файл aligned.sam
!bwa mem ecoli.fna clean_1.fastq clean_2.fastq > aligned.sam
#Преобразование SAM → BAM → сортировка
!samtools view -S -b aligned.sam > aligned.bam
!samtools sort aligned.bam -o aligned_sorted.bam
!samtools index aligned_sorted.bam


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bwa is already the newest version (0.7.17-6).
samtools is already the newest version (1.13-4).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
[bwa_index] Pack FASTA... 0.05 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.48 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.03 sec
[bwa_index] Construct SA from BWT and Occ... 0.72 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index ecoli.fna
[main] Real time: 2.464 sec; CPU: 2.313 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 70692 sequences (10000074 bp)...
[M::process] read 70688 sequences (10000202 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (117, 31403, 11, 113)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (462, 901, 2491)
[M::mem_pestat] 

In [ ]:
#Проверка. Покажет количество выровненных ридов, процент покрытия и др
!samtools flagstat aligned_sorted.bam

2623584 + 0 in total (QC-passed reads + QC-failed reads)
2602652 + 0 primary
0 + 0 secondary
20932 + 0 supplementary
0 + 0 duplicates
0 + 0 primary duplicates
2475989 + 0 mapped (94.37% : N/A)
2455057 + 0 primary mapped (94.33% : N/A)
2602652 + 0 paired in sequencing
1301326 + 0 read1
1301326 + 0 read2
2375808 + 0 properly paired (91.28% : N/A)
2437740 + 0 with itself and mate mapped
17317 + 0 singletons (0.67% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


Очень высокий процент выравнивания (~94%) — отличное качество данных!

 Более 91% ридов выровнены в парах правильно — тоже отлично.

 Singletons — один рид из пары выровнен, другой — нет. У нас таких всего ~0.67%.

 Данные качественные:

Почти все риды выровнены.

Высокий процент правильно спаренных ридов.

Нет дубликатов или химерных пар.

In [ ]:
!samtools view aligned_sorted.bam | head

SRR2584863.72312	2147	NC_000913.3	1	60	116H34M	=	4641543	4641652	AGCTTTTCATTCTGACTGCAACGGGCAATATGTC	DDDADDDDDEDDEEEDDDDDCDDBDB<?CCDDDD	NM:i:0	MD:Z:34	MC:Z:110M40H	AS:i:34	XS:i:0	SA:Z:NC_000913.3,4641537,+,116M34S,60,1;
SRR2584863.236239	99	NC_000913.3	1	60	55S95M	=	305	454	GCATGATATTGAAAAAAATATCACCAAATAAAAAACGCCTTAGTAAGTATTTTTCAGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAG	@BCFFFFFHHHHHJJJJJIIJJIJJJJIIJJJJJJIGJIJIIIAHGHCFGIJJJIIIHHHHHGHFFFFFBDECACCDDDB=BBCDDEDDEDCDCDDDB>@@>CDDDDBBDCDDACC@@CCCCC:<A@CCCD@@>C>CC@C>A4<8099?B	NM:i:0	MD:Z:95	MC:Z:150M	AS:i:95	XS:i:0	SA:Z:NC_000913.3,4641598,+,55M95S,60,0;
SRR2584863.275624	99	NC_000913.3	1	60	14S136M	=	145	294	AGTAAGTATTTTTCAGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAA	@B@FFFBDHHHHHJJIGJJJJJJJJJJJJIJJJJJJIJJJIJIGIGJJJJJJIJFGIHIJJJJJIJHFFFDFEEEEEEEDDDDDDDDDDEDDDDDCDDDDDCBDDBDDDCDDDEEDDDDEDDDEDDDDDDDDDDDDD

In [ ]:
# Более подробная статистика (глубина, GC-контент, распределение длин и т.д.).
!samtools stats aligned_sorted.bam > alignment_stats.txt


In [ ]:
#Cтатистика по каждому контигу (или хромосоме): число выровненных и невыравненных ридов.
!samtools idxstats aligned_sorted.bam


NC_000913.3	4641652	2475989	17317
*	0	0	130278
